# Example notebook with custom R:R stoploss and takeprofit based on ATR

## With use of SQLite3 datase to create a dataframe



In [12]:
# Load libraries
import pandas as pd
import pandas_ta as pta
import talib.abstract as ta
from pandas import DataFrame
import numpy as np
import mplfinance as mpf
import sqlite3

In [13]:
# Connect to database and create a dataframe from last 999 rows (Descending)
conn = sqlite3.connect("file:../data/BTCUSDT.db?mode=ro", uri=True)
# dataframe = pd.read_sql_query("SELECT * FROM _1d ORDER BY openTime DESC LIMIT 999", conn)
# dataframe = pd.read_sql_query("SELECT openTime,open,high,low,close,volume,dow,color,humandate FROM _1d ORDER BY openTime DESC LIMIT 999", conn)

# This is the correct querie
dataframe = pd.read_sql_query("SELECT * FROM (SELECT openTime,open,high,low,close,volume,dow,color,humandate FROM _1d ORDER BY openTime DESC LIMIT 100) sub ORDER BY openTime ASC", conn)

conn.close()

# Create human readable time
dataframe['openTime']=(pd.to_datetime(dataframe['openTime'],unit='s'))

# Set openTime to the index, reorder the complete dataframe and add it to a new df for further analysis
# NOTE: after the new qierie, there is actually no need for the line below. Just keep this as reference...
# df = dataframe.set_index(['openTime']).sort_values(by=['openTime'], ascending=True)

In [14]:
# Show the new df
dataframe

,openTime,open,high,low,close,volume,dow,color,humandate
0,2022-02-26,39219.16,40348.45,38573.18,39116.72,29361.25680,Saturday,Red,20220226
1,2022-02-27,39116.73,39855.70,37000.00,37699.07,46229.44719,Sunday,Red,20220227
2,2022-02-28,37699.08,44225.84,37450.17,43160.00,73945.63858,Monday,Green,20220228
3,2022-03-01,43160.00,44949.00,42809.98,44421.20,61743.09873,Tuesday,Green,20220301
4,2022-03-02,44421.20,45400.00,43334.09,43892.98,57782.65081,Wednesday,Red,20220302
...,...,...,...,...,...,...,...,...,...
95,2022-06-01,31801.05,31982.97,29301.00,29805.83,103395.63382,Wednesday,Red,20220601
96,2022-06-02,29805.84,30689.00,29594.55,30452.62,56961.42928,Thursday,Green,20220602
97,2022-06-03,30452.63,30699.00,29282.36,29700.21,54067.44727,Friday,Red,20220603
98,2022-06-04,29700.21,29988.88,29485.00,29864.04,25617.90113,Saturday,Green,20220604


In [15]:
# Show latest entries
dataframe[['open','high','low','close','volume','color']].tail(5)

,open,high,low,close,volume,color
95,31801.05,31982.97,29301.00,29805.83,103395.63382,Red
96,29805.84,30689.00,29594.55,30452.62,56961.42928,Green
97,30452.63,30699.00,29282.36,29700.21,54067.44727,Red
98,29700.21,29988.88,29485.00,29864.04,25617.90113,Green
99,29864.03,30189.00,29531.42,29919.21,23139.92810,Green


## Creating indicators

In [16]:
# MACD indicator
macd = ta.MACD(
        dataframe,
        fastperiod=12,
        fastmatype=0,
        slowperiod=26,
        slowmatype=0,
        signalperiod=9,
        signalmatype=0,
    )
dataframe["macd"] = macd["macd"]
dataframe["macdsignal"] = macd["macdsignal"]
dataframe["macdhist"] = macd["macdhist"]

# ATR
dataframe['atr'] = ta.ATR(dataframe, timeperiod=14)
dataframe[['open','high','low','close','volume','atr','macd','macdsignal','macdhist']]

,open,high,low,close,volume,atr,macd,macdsignal,macdhist
0,39219.16,40348.45,38573.18,39116.72,29361.25680,NaN,NaN,NaN,NaN
1,39116.73,39855.70,37000.00,37699.07,46229.44719,NaN,NaN,NaN,NaN
2,37699.08,44225.84,37450.17,43160.00,73945.63858,NaN,NaN,NaN,NaN
3,43160.00,44949.00,42809.98,44421.20,61743.09873,NaN,NaN,NaN,NaN
4,44421.20,45400.00,43334.09,43892.98,57782.65081,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,31801.05,31982.97,29301.00,29805.83,103395.63382,1772.248263,-1130.837245,-1652.239059,521.401814
96,29805.84,30689.00,29594.55,30452.62,56961.42928,1723.834102,-1024.550333,-1526.701314,502.150980
97,30452.63,30699.00,29282.36,29700.21,54067.44727,1701.891666,-989.622635,-1419.285578,429.662942
98,29700.21,29988.88,29485.00,29864.04,25617.90113,1616.319404,-937.910819,-1323.010626,385.099807
